In [160]:
import pandas as pd
import numpy as np
import requests
import json
import csv
import time
import datetime
import math

In [161]:
my_client_id = 'V5VI7rbXYAHpaQ'
my_client_secret = 'wMX4ps5HWnRWYgzTfm-2tSBhP9c'
my_user_agent = 'Data for research'

import praw
from psaw import PushshiftAPI
reddit = praw.Reddit(client_id=my_client_id, client_secret=my_client_secret, user_agent=my_user_agent)
api = PushshiftAPI(reddit)

In [162]:
def getPushshiftData(query, after, before, sub):
    url = 'https://api.pushshift.io/reddit/search/submission/?title='+str(query)+'&size=10000&after='+str(after)+'&before='+str(before)+'&subreddit='+str(sub)
    print(url)
    r = requests.get(url)
    data = json.loads(r.text)
    return data['data']

In [163]:
def collectSubData(subm):
    subData = list() #list to store data points
    title = subm['title']
    url = subm['url']
    try:
        flair = subm['link_flair_text']
    except KeyError:
        flair = "NaN"    
    author = subm['author']
    sub_id = subm['id']
    score = subm['score']
    created = datetime.datetime.fromtimestamp(subm['created_utc']) #1520561700.0
    numComms = subm['num_comments']
    permalink = subm['permalink']
    
    subData.append((sub_id,title,url,author,score,created,numComms,permalink,flair))
    subStats[sub_id] = subData

In [164]:
#Subreddit to query
sub='conspiracy'
#before and after dates
before = "1594669044" #13/07
#before = math.ceil(datetime.utcnow().timestamp())
after = "1594166400"  #08/07 
query = "Wayfair"
subCount = 0
subStats = {}

In [165]:
data = getPushshiftData(query, after, before, sub)# Will run until all posts have been gathered 
# from the 'after' date up until before date
while len(data) > 0:
    for submission in data:
        collectSubData(submission)
        subCount+=1
    # Calls getPushshiftData() with the created date of the last submission
    print(len(data))
    print(str(datetime.datetime.fromtimestamp(data[-1]['created_utc'])))
    after = data[-1]['created_utc']
    data = getPushshiftData(query, after, before, sub)
    
print(len(data))

https://api.pushshift.io/reddit/search/submission/?title=Wayfair&size=10000&after=1594166400&before=1594669044&subreddit=conspiracy
100
2020-07-10 22:08:08
https://api.pushshift.io/reddit/search/submission/?title=Wayfair&size=10000&after=1594433288&before=1594669044&subreddit=conspiracy
100
2020-07-11 08:35:58
https://api.pushshift.io/reddit/search/submission/?title=Wayfair&size=10000&after=1594470958&before=1594669044&subreddit=conspiracy
100
2020-07-11 14:41:33
https://api.pushshift.io/reddit/search/submission/?title=Wayfair&size=10000&after=1594492893&before=1594669044&subreddit=conspiracy
100
2020-07-11 23:12:48
https://api.pushshift.io/reddit/search/submission/?title=Wayfair&size=10000&after=1594523568&before=1594669044&subreddit=conspiracy
100
2020-07-12 18:33:41
https://api.pushshift.io/reddit/search/submission/?title=Wayfair&size=10000&after=1594593221&before=1594669044&subreddit=conspiracy
62
2020-07-13 15:26:26
https://api.pushshift.io/reddit/search/submission/?title=Wayfair&

In [166]:
print(str(len(subStats)) + " submissions have added to list")
print("1st entry is:")
print(list(subStats.values())[0][0][1] + " created: " + str(list(subStats.values())[0][0][5]))
print("Last entry is:")
print(list(subStats.values())[-1][0][1] + " created: " + str(list(subStats.values())[-1][0][5]))

562 submissions have added to list
1st entry is:
Is wayfair selling children ? created: 2020-07-09 01:45:10
Last entry is:
Videoclip of what happens in your wayfair cart. created: 2020-07-13 15:26:26


In [167]:
def updateSubs_file():
    upload_count = 0
    location = "C:\\Users\\posit\\Downloads"
    print("input filename of submission file, please add .csv")
    filename = input()
    file = location + filename
    with open(file, 'w', newline='', encoding='utf-8') as file: 
        a = csv.writer(file, delimiter=',')
        headers = ["Post ID","Title","Url","Author","Score","Publish Date","Total No. of Comments","Permalink","Flair"]
        a.writerow(headers)
        for sub in subStats:
            a.writerow(subStats[sub][0])
            upload_count+=1
            
        print(str(upload_count) + " submissions have been uploaded")
        
updateSubs_file()

input filename of submission file, please add .csv
wayfair_submissions.csv
562 submissions have been uploaded


In [168]:
df = pd.read_csv("C:\\Users\\posit\\Downloadswayfair_submissions.csv")
df.head()

,Post ID,Title,Url,Author,Score,Publish Date,Total No. of Comments,Permalink,Flair
0,hnxnn5,Is wayfair selling children ?,https://i.redd.it/9rnx5ml5sr951.jpg,pawsfurdaze,1,2020-07-09 01:45:10,59,/r/conspiracy/comments/hnxnn5/is_wayfair_selli...,NaN
1,ho2pqv,Is it possible Wayfair involved in Human traff...,https://i.redd.it/ch6ep77kyt951.jpg,PrincessPeach1987,1,2020-07-09 09:04:38,2494,/r/conspiracy/comments/ho2pqv/is_it_possible_w...,NaN
2,hoay8h,Pt. 2: from my recent post I showed you one sc...,https://i.redd.it/5mkghvpm8w951.jpg,oLoLoS,2,2020-07-09 16:44:38,10,/r/conspiracy/comments/hoay8h/pt_2_from_my_rec...,NaN
3,hoch8y,The Wayfair 'IMG USA' search works with lots o...,https://www.reddit.com/r/conspiracy/comments/h...,nova-north,3,2020-07-09 18:07:47,44,/r/conspiracy/comments/hoch8y/the_wayfair_img_...,NaN
4,hojt15,wayfair sells children (proof + true),https://www.reddit.com/r/conspiracy/comments/h...,wake_up_its_here,1,2020-07-10 02:15:44,0,/r/conspiracy/comments/hojt15/wayfair_sells_ch...,New User


In [169]:
submissions = []
comments = []

In [170]:
for i in df['Post ID']:
    author = []
    parent = []
    body = []
    date = []
    response = []
    submission = reddit.submission(id=f'{i}')
    submissions.append(submission)
    submission.comments.replace_more(limit=None)
    for comment in submission.comments.list():
        comments.append(comment)

In [171]:
len(comments)

16753

In [172]:
author = []
body = []
date = []
link_id = []
parent_id = []
sub_com = []
replies = []
date_rep = []
author_rep = []
link_rep = []
parent_rep = []
sub_rep = []
comments_df = pd.DataFrame(columns=['author', 'body', 'created_utc','link_id','parent_id','submission'])
rep_df = pd.DataFrame(columns=['author', 'body', 'created_utc','link_id','parent_id','submission'])

In [173]:
for i in range(len(comments)):
    author.append(comments[i].author)
    body.append(comments[i].body)
    date.append(comments[i].created_utc)
    link_id.append(comments[i].link_id)
    parent_id.append(comments[i].parent())
    sub_com.append(comments[i].submission)
    for reply in comments[i].replies:
        replies.append(reply.body)
        date_rep.append(reply.created_utc)
        author_rep.append(reply.author)
        link_rep.append(reply.link_id)
        parent_rep.append(reply.parent())
        sub_rep.append(reply.submission)

In [297]:
author_df = pd.DataFrame(author)
author_df['index'] = author_df.index
comment_df = pd.DataFrame(body)
comment_df['index'] = comment_df.index
date_df = pd.DataFrame(date)
date_df['index'] = date_df.index
link_df = pd.DataFrame(link_id)
link_df['index'] = link_df.index
parent_df = pd.DataFrame(parent_id)
parent_df['index'] = parent_df.index
sub_df = pd.DataFrame(sub_com)
sub_df['index'] = sub_df.index

In [310]:
author_rep_df = pd.DataFrame(author_rep)
author_rep_df['index'] = author_rep_df.index
comment_rep_df = pd.DataFrame(replies)
comment_rep_df['index'] = comment_rep_df.index
date_rep_df = pd.DataFrame(date_rep)
date_rep_df['index'] = date_rep_df.index
link_rep_df = pd.DataFrame(link_rep)
link_rep_df['index'] = link_rep_df.index
parent_rep_df = pd.DataFrame(parent_rep)
parent_rep_df['index'] = parent_rep_df.index
sub_rep_df = pd.DataFrame(sub_rep)
sub_rep_df['index'] = sub_rep_df.index

In [298]:
comments_df = pd.DataFrame()
comments_df['index'] = np.arange(0,16753,1)

In [299]:
comments_df = comments_df.join(author_df,on='index',how='left',rsuffix='_right')

In [300]:
comments_df = comments_df.join(comment_df,on='index',how='left',rsuffix='_right')

In [301]:
comments_df = comments_df.join(date_df,on='index',how='left',rsuffix='_right')

In [302]:
comments_df = comments_df.join(link_df,on='index',how='left',rsuffix='_right')

In [303]:
comments_df = comments_df.join(parent_df,on='index',how='left',rsuffix='_right')

In [304]:
comments_df = comments_df.join(sub_df,on='index',how='left',rsuffix='_right')

In [305]:
comments_df.columns

Index(['index', '0', 'index_right', '0_right', 'index_right', '0',
       'index_right', '0_right', 'index_right', '0', 'index_right', '0_right',
       'index_right'],
      dtype='object')

In [306]:
comments_df.drop(['index_right','index'], axis=1,inplace=True)

In [308]:
comments_df.columns = ['author', 'comment','date','link','parent','submission']

In [309]:
comments_df.head()

,author,comment,date,link,parent,submission
0,AutoModerator,###[Meta] Sticky Comment\n\n[Rule 2](https://w...,1.594274e+09,t3_hnxnn5,hnxnn5,hnxnn5
1,That_Weird_Girl,"Kinda reminds me of ""hidden links"" that chines...",1.594282e+09,t3_hnxnn5,hnxnn5,hnxnn5
2,4NKALI0N,The throw pillows and cabinets are literally n...,1.594407e+09,t3_hnxnn5,hnxnn5,hnxnn5
3,sorryimveryhigh,if so imagine being the guy who splurges and s...,1.594274e+09,t3_hnxnn5,hnxnn5,hnxnn5
4,ThePiz91,Good post op!! Not really proving anything but...,1.594277e+09,t3_hnxnn5,hnxnn5,hnxnn5


In [314]:
replies_df = pd.DataFrame()
replies_df['index'] = np.arange(0,11251,1)

In [318]:
replies_df = replies_df.join(author_rep_df,on='index',how='left',rsuffix='_right')
replies_df = replies_df.join(comment_rep_df,on='index',how='left',rsuffix='_right')
replies_df = replies_df.join(date_rep_df,on='index',how='left',rsuffix='_right')
replies_df = replies_df.join(link_rep_df,on='index',how='left',rsuffix='_right')
replies_df = replies_df.join(parent_rep_df,on='index',how='left',rsuffix='_right')
replies_df = replies_df.join(sub_rep_df,on='index',how='left',rsuffix='_right')

In [319]:
replies_df.columns

Index(['index', '0', 'index_right', '0_right', 'index_right', '0',
       'index_right', '0_right', 'index_right', '0', 'index_right', '0_right',
       'index_right'],
      dtype='object')

In [320]:
replies_df.drop(['index_right','index'], axis=1,inplace=True)

In [321]:
replies_df.columns = ['author', 'comment','date','link','parent','submission']

In [322]:
replies_df.head()

,author,comment,date,link,parent,submission
0,Maschinenherz,"Channel ""Survigin life"" on youtube discovered ...",1.594485e+09,t3_hnxnn5,fxeidw1,hnxnn5
1,pawsfurdaze,We’ll probably they don’t ship an actual cabin...,1.594274e+09,t3_hnxnn5,fxe9du6,hnxnn5
2,mars42069,This is gaining traction so they likely saw so...,1.594356e+09,t3_hnxnn5,fxedabx,hnxnn5
3,Flexorcist,If you're spending that kind of money on a sto...,1.594274e+09,t3_hnxnn5,fxe9dpw,hnxnn5
4,pawsfurdaze,"Sometimes though, they’ll be the same exact sh...",1.594274e+09,t3_hnxnn5,fxe9dpw,hnxnn5


In [324]:
all_com_rep = comments_df.append(replies_df)

In [ ]:
all_com_rep.to_excel("wayfair.xlsx")